In [1]:
from bs4 import BeautifulSoup
import os
import re
import csvtools
import pandas as pd
from nltk import word_tokenize
import string
import pymorphy2
import bs4
import collections
from collections import Counter
import nltk
import nltk.tokenize.punkt
from nltk.collocations import *
import requests
from datetime import datetime
from prozhitotools import dump
from tqdm.auto import tqdm
import ipywidgets
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
morph = pymorphy2.MorphAnalyzer()
extended_punctuation = string.punctuation + '\'\'" №~—»«..``’.--–“”―…"''" '
from stop_words import get_stop_words
stoplist = get_stop_words('ru') + ['среди', 'аа', 'некий', 'либо', 'видимо', 'итак', 'словно', 'наверное', 'возможно', '-й', '-х', 'например', 'что-нибудь', 'вместо', 'кто-то', 'несмотря', 'коли', 'безъ', 'нѣтъ', 'всѣхъ', 'поэтому', 'почему-то', 'ибо', 'из-за', 'можетъ', 'неужели', 'дѣть', 'намъ', 'оттого', 'чрез', 'вероятно', 'многий', 'ах', 'пред', 'пусть', 'нынче', 'б', 'дабы', 'чрезвычайно', 'разумеется', 'вовсе', 'кн', 'гр', 'св', '-го', 'кой', 'франц.', 'оный', 'сей', 'свой', 'едва', 'самый', 'го', 'мол', 'час', 'часы', 'ть', 'минута', 'метр', 'г.', 'ко', 'въ', 'съ', 'мнѣ', 'къ', 'какъ', 'отъ', 'такъ', 'ежели', 'себѣ', 'былъ', 'онъ', 'томъ', 'сие', 'нрзб', 'изъ', 'тѣмъ', 'ихъ', 'чѣмъ', 'всѣ'] + ['com', 'id', 'de', 'la', 'et', 'le', 'que', 'je', 'il', 'est', 'les', 'un']

In [2]:
import sys
import wheel

Загружаем дневниковые записи из дампа «Прожито» и таблицу с метаданными

In [3]:
dw = dump.Wrapper(csvpath='/Users/victoriavrbv/Documents/Diploma/prozhito-dump-1')

In [4]:
authors_data = pd.read_csv('/Users/victoriavrbv/Documents/Diploma/Prozhito-people - person_data.csv')

In [5]:
authors_data

,id,first_name,patronymic,family_name,gender,birth_date,death_date,edition,additional_info,from_natasha,from_tags,info,nickname,wiki
0,3739,Чудинова,Александра,Дмитриевна,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
1,3740,Аборкин,Дмитрий,Георгиевич,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
2,3741,ТИХОНОВ,СЕРГЕЙ,Константинович,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
3,3742,Митрофанов,Владимир,Петрович,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
4,3743,Манский,Давыд,Моисеевич,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28980,28808,с,.,найденов,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28981,28874,Иоганна,Георга,Корба,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28982,28809,л,.,авилова,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28983,29003,де,-,жимель,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN


In [6]:
#считаем сколько мужчин и женщин в базе
authors_data.groupby('gender')['id'].nunique()

gender
0.0     810
1.0    2882
Name: id, dtype: int64

Создаём нужные функции для предобработки данных

In [7]:
#функция для лемматизации
def preprocess_text(some_text):
    '''
    берет обычный текст, возвращает лемматизированный
    '''    
    some_text = re.sub(r'\[|\]|\\|([0-9])|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ~]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist and morph.parse(word)[0].normal_form not in extended_punctuation]
    return lemmatized_text

In [8]:
stoplist2 = get_stop_words('ru') + ['фр', 'франц.', 'что', 'оный', 'сей', 'т.е', 'свой', 'едва', 'самый', 'го', 'мол', 'час', 'часы', 'ть', 'минута', 'метр', 'г.', 'ко', 'въ', 'съ', 'мнѣ', 'къ', 'какъ', 'отъ', 'такъ', 'ежели', 'себѣ', 'былъ', 'онъ', 'томъ', 'сие', 'нрзб', 'изъ', 'тѣмъ', 'ихъ', 'чѣмъ', 'всѣ'] + ['com', 'id', 'de', 'la', 'et', 'le', 'que', 'je', 'il', 'est', 'les', 'un']
names_list = ["владимир", "аля", "лёля", "ася", "ник", "никола", "юрий", "борис", "виктор", "андрей", "ирина", "варя", "валя", "анович", "шура", "боря", "шурочка", "леонид", "вера", "зуля", "зося", "михаилович", "николаевич", "наташа", "митя", "ляля", "иванович", "зина", "володя", "никита", "лена", "коля", "николай", "маруся", "лиза", "олег", "вася", "игорь", "георгий", "львович", "марина", "софья", "андреевич", "александр", "цветаев", "владимирович", "таня", "анна", "василиевич", "петрович", "сергей", "василиевич", "саша", "павлович", "маша", "миша", "константин", "пётр", "павел", "лена", "лев", "серёжа", "оля", "яковлевич", "нина", "шурик", "женя", "соня", "семён", "костя"]
def preprocess_text_for_topics(some_text):
    '''
    берет обычный текст, возвращает лемматизированный
    '''    
    some_text = re.sub(r'\[|\]|\\|([0-9])|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ~]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist2 and morph.parse(word)[0].normal_form not in extended_punctuation]
    return lemmatized_text

In [4]:
def flatten_list(_2d_list):
    '''
    берет вложенный список, возвращает плоский
    '''
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [7]:
len(dw.notes)

384587

In [153]:
#отделяем записи 17-19 века
war_records = []
period = dw.notes[(1942, 1, 1) : (1945, 12, 31)]
for item in period:
    war_records.append(item.text)    

In [154]:
with open('one_more_war_records.txt', 'w') as f:
    for item in war_records:
        f.write("%s\n" % item)

In [3]:
eight = dw.notes[(1701, 1, 1) : (1800, 12, 31)]

In [4]:
len(eight)

623

In [327]:
#отделяем записи 20 века
twenty_cent = []
twenty = dw.notes[(1960, 1, 1) : (1970, 12, 31)]
#twenty_2 = dw.notes[(1948, 1, 1) : (1980, 12, 31)]
for item in twenty:
    twenty_cent.append(item.text) 

In [25]:
#ищем самых популярных авторов периода
authors4 = []
records = dw.notes[(1948, 1, 1) : (1979, 12, 31)]
for item in records:
    authors4.append(item.author)

In [26]:
authors4 = list(Counter(authors4).most_common())
authors_4 = pd.DataFrame(authors4)
#authors_5.columns = ["Author", "1"]
#len(authors_1["Author"])

In [27]:
authors_4

,0,1
0,#318 Владимир Афанасьевич Швец,6295
1,#129 Михаил Яковлевич Гробман,3053
2,#671 Николай Алексеевич Соколов,2983
3,#386 Борис Иванович Вронский,2541
4,#2258 Олег Владимирович Амитров,2472
...,...,...
276,#354 Екатерина Дмитриевна Воейкова-Ильина,1
277,#663 Раиса Ширинская,1
278,#525 Аркадий Поздняков,1
279,#1794 Семён Борисович Ласкин,1


In [426]:
#ищем записи определенных авторов
amitrov_texts = []
kozakov_texts = []
shvets_texts = []
grobman_texts = []
sokolov_texts = []
records = dw.notes[(1948, 1, 1) : (1979, 12, 31)]
for item in records:
    author_kozakov = re.search(r"Николай Николаевич Козаков", str(item.author))
    author_amitrov = re.search(r"Олег Владимирович Амитров", str(item.author))
    author_shvets = re.search(r"Владимир Афанасьевич Швец", str(item.author))
    author_grobman = re.search(r"Михаил Яковлевич Гробман", str(item.author))
    author_sokolov = re.search(r"Николай Алексеевич Соколов", str(item.author))
    if author_kozakov:
        kozakov_texts.append(item.text)
    if author_amitrov:
        amitrov_texts.append(item.text)
    if author_shvets:
        shvets_texts.append(item.text)
    if author_grobman:
        grobman_texts.append(item.text)
    if author_sokolov:
        sokolov_texts.append(item.text)

In [16]:
#считаем общее количество записей в периоде
period_1 = dw.notes[(1799, 1, 1) : (1898, 12, 31)]
period_2 = dw.notes[(1899, 1, 1) : (1938, 12, 31)]
period_3 = dw.notes[(1939, 1, 1) : (1947, 12, 31)]
period_4 = dw.notes[(1948, 1, 1) : (1979, 12, 31)]
period_5 = dw.notes[(1980, 1, 1) : (2017, 12, 31)]
#period_6 = dw.notes[(1960, 1, 1) : (1979, 12, 31)]
#period_7 = dw.notes[(1980, 1, 1) : (1998, 12, 31)]
#period_8 = dw.notes[(1999, 1, 1) : (2017, 12, 31)]

In [17]:
diaries1_all = {}
for i, diaries in enumerate(period_1):
    diaries1_all[period_1[i].ID] = [period_1[i].text]

In [18]:
diaries2_all = {}
for i, diaries in enumerate(period_2):
    diaries2_all[period_2[i].ID] = [period_2[i].text]
diaries3_all = {}
for i, diaries in enumerate(period_3):
    diaries3_all[period_3[i].ID] = [period_3[i].text]
diaries4_all = {}
for i, diaries in enumerate(period_4):
    diaries4_all[period_4[i].ID] = [period_4[i].text]
diaries5_all = {}
for i, diaries in enumerate(period_5):
    diaries5_all[period_5[i].ID] = [period_5[i].text]

In [67]:
period = dw.notes[(1799, 1, 1) : (2017, 12, 31)]

In [146]:
len(diaries5_all)

53151

In [329]:
#считаем количество токенов в периоде 
period4_tokenized = []
for item in twenty_cent:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            period4_tokenized.append(token)
#len(period1_tokenized)

In [330]:
len(period4_tokenized)

4304675

In [24]:
with open('old_records.txt', 'w') as f:
    for item in old_records:
        f.write("%s\n" % item)

Собираем корпус записей с помощью регулярных выражений

In [5]:
diaries_dic = {}
info = {}
info_list = []
meta_info = pd.DataFrame(columns = ['ID записи', 'Автор', 'ID дневника', 'Дата'])
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?влюб(ился|илась|лен(\s|а|ный|ная))", dw.notes[i].text.lower()):
    #if re.search(r"влюб(ленност.{1,2}|ился|илась|лен(\s|а|ный|ная))", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info) 

In [6]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    #if re.search(r"(он|она|я)\sизмени(л|ла)\s", dw.notes[i].text):
    if re.search(r"(я\s([а-яё-]+?\s)?разлюби(|л|ла))|(разлюби(л|ла|шь)\sменя)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
for i, diaries in enumerate(dw.notes):
    if re.search(r"(он|она|я|ты)\sизмени(л|ла)\s(мне)?", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?ревн(ую|овал|овала|ивый|ивая|ивец|ивица)", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)

In [7]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"ревн(овал|овала|уешь|ует)\sменя", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)

In [8]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"(любо?в.{0,2}\s(мо.{1,2}|наш.{1,2}|сво.{1,2}|его|её|ее|тво.{1,2}))|((мо.{1,2}|наш.{1,2}|сво.{1,2}|его|ее|её|тво.{1,2})\sлюбо?в.{0,2})", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)  

In [9]:
 for i, diaries in enumerate(dw.notes):
    if re.search(r"любим.{0,3}\sм{2,4}|м{2,4}\sлюбим.{0,3}|я\sлюбим.{0,2}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)  

In [10]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?(по)?люб(лю|я|ив|ил|ила|)\s(вас|ваш|тебя|твой|его|ее|её)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [11]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"(ты|он|она)\s(по|не\s)?люб(ишь|я|ив|ил|ила|ит)\sменя", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)

In [12]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"я?\s([а-яё-]+?\s)?(вас|тебя|меня)\s([а-яё,\s-]+?)?(по)?люб(лю|я|ив|ил|ила|ишь)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [13]:
diaries_dic_again = {}
diaries_dic_more = {}
for i, diaries in enumerate(dw.notes):
    #if re.search(r"поцел(овал|овала|уй)\sменя", dw.notes[i].text.lower()):
    if re.search(r"перв.{1,3}поцелу.{1,3}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
    #if re.search(r"настоящ.{2,3}\sлюбо?в{1,2}", dw.notes[i].text.lower()):
    #    diaries_dic_more[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [14]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"(призна(лся|лась)\s(мне\sв\sлюбви|в\sлюбви\sмне))|(признани.{1,2}\sв\sлюбви)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [15]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"(мо.{1,3}\sволокитств.{1,2})|(волочи(лся|ться)\sза\sмной)|(я\s([а-яё-]+?\s)?волочи(лся|ться))", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [16]:
diaries_dic_again = {}
diaries_dic_more = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"любим.{2,3}\s([а-яё-]+?\s)?человек.{0,2}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [17]:
len(diaries_dic)

6283

In [154]:
#отбираем записи 20 века
twenty_cent = []
for item in diaries_dic.items():
    if re.search("\(19\d{2},\s\d{1,2},\s\d{1,2}\)", str(item[0])):
    #if key.startswith('(1910'):
        twenty_cent.append(item)
#[key for key, value in diaries_dic if key.startswith('(1910')]

In [254]:
#results

In [23]:
#ищем повторяющиеся записи в метаданных
pd.concat(g for _, g in meta_info.groupby("ID") if len(g) > 1)

,ID,Автор,ID дневника,Дата
1480,1471,0,2,"(1917, 2, 14)"
4291,1471,0,2,"(1917, 2, 14)"
84,2404,#13 Александр Александрович Блок,10,"(1911, 12, 1)"
5652,2404,#13 Александр Александрович Блок,10,"(1911, 12, 1)"
118,2869,#13 Александр Александрович Блок,10,"(1920, 11, 7)"
...,...,...,...,...
6947,467461,#1478 Нелли,1488,"(1917, 9, 25)"
1725,468335,#3334 Ольга Ивановна Баранова,3344,"(1923, 1, 21)"
5835,468335,#3334 Ольга Ивановна Баранова,3344,"(1923, 1, 21)"
199,468933,#3477 Эра Гансовна Мундецем,3487,"(1944, 6, 8)"


In [18]:
#убираем их
meta_info_cleaned = meta_info.drop_duplicates(subset=['ID'])
meta_info_cleaned

,ID,Автор,ID дневника,Дата
0,140292,#507 Степан Петрович Жихарев,515,"(1805, 7, 20)"
1,444870,#719 Николай Дмитриевич Дурново,729,"(1812, 1, 22)"
2,178659,#763 Анна Алексеевна Андро,773,"(1828, 10, 6)"
3,171755,#706 Алексей Николаевич Вульф,716,"(1831, 2, 10)"
4,442455,#790 Дарья Фёдоровна Фикельмон,800,"(1831, 7, 5)"
...,...,...,...,...
7136,435616,#1949 Юрий Михайлович Кублановский,1959,"(2010, 3, 14)"
7137,394573,#2338 Ирина Львовна Шолохова,2348,"(2011, 1, 3)"
7139,394581,#2338 Ирина Львовна Шолохова,2348,"(2012, 12, 2)"
7140,394582,#2338 Ирина Львовна Шолохова,2348,"(2012, 12, 29)"


In [19]:
#превращаем дату в datetime object
#meta_info_cleaned.info()
#meta_info_cleaned['Автор']= meta_info_cleaned['Автор'].astype(str)
#meta_info_cleaned['Дата']= meta_info_cleaned['Дата'].astype('datetime64[ns]')
meta_info_cleaned['Дата'] = pd.to_datetime(meta_info_cleaned['Дата'], format='(%Y, %m, %d)')
meta_info_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6283 entries, 0 to 7142
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           6283 non-null   int64         
 1   Автор        6280 non-null   object        
 2   ID дневника  6283 non-null   int64         
 3   Дата         6283 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 245.4+ KB


/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_25144/2614964356.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned['Дата'] = pd.to_datetime(meta_info_cleaned['Дата'], format='(%Y, %m, %d)')


In [86]:
#meta_1 771 запись -- 46107 записей всего
#meta_2 1437 записей -- 79599 записей всего
#meta_3 805 записей -- 50995 записей всего
#meta_4 1186 записей -- 68083 записей всего
#meta_5 468 записей -- 31066 записей всего
#meta_6 635 записей -- 47007 записей всего
#meta_7 747 записи -- 40143 записей всего
#meta_8 356 записей -- 13008 записей всего

In [20]:
#отбираем записи определенного периода
meta_1 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1798-12-31') & (meta_info_cleaned['Дата'] <= '1898-12-31')]
meta_2 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1898-12-31') & (meta_info_cleaned['Дата'] <= '1938-12-31')]
meta_3 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1938-12-31') & (meta_info_cleaned['Дата'] <= '1947-12-31')]
meta_4 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1947-12-31') & (meta_info_cleaned['Дата'] <= '1979-12-31')]
meta_5 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1979-12-31') & (meta_info_cleaned['Дата'] <= '2017-12-31')]

In [622]:
#len(meta_1) #818 записи -- 50467 записей всего 0.016
#len(meta_2) #2207 записи -- 130594 записей всего 0.017
#len(meta_3) #1301 записей -- 73111 записей всего 0.018
#len(meta_4) #936 записи -- 73045 записей всего 0.013
#len(meta_5) #1017 записи -- 53151 записей всего 0.019

1017

In [21]:
#отбираем ID записей определенного периода
period1_ids = []
period2_ids = []
period3_ids = []
period4_ids = []
period5_ids = []
for item in meta_1["ID"]:
    period1_ids.append(item)
for item in meta_2["ID"]:
    period2_ids.append(item)
for item in meta_3["ID"]:
    period3_ids.append(item)
for item in meta_4["ID"]:
    period4_ids.append(item)
for item in meta_5["ID"]:
    period5_ids.append(item)

In [22]:
#собираем записи разных периодов в списки
period1_records = []
period2_records = []
period3_records = []
period4_records = []
period5_records = []
for item in diaries_dic.items():
    for x in period1_ids:
        if item[0] == x:
            period1_records.append(item[1])
    for x in period2_ids:
        if item[0] == x:
            period2_records.append(item[1])
    for x in period3_ids:
        if item[0] == x:
            period3_records.append(item[1])
    for x in period4_ids:
        if item[0] == x:
            period4_records.append(item[1])
    for x in period5_ids:
        if item[0] == x:
            period5_records.append(item[1])

In [23]:
records5_all = []
records4_all = []
records3_all = []
records2_all = []
records1_all = []
for item in diaries5_all.items():
    records5_all.append(item[1])
for item in diaries4_all.items():
    records4_all.append(item[1])
for item in diaries3_all.items():
    records3_all.append(item[1])
for item in diaries2_all.items():
    records2_all.append(item[1])
for item in diaries1_all.items():
    records1_all.append(item[1])

NameError: name 'diaries5_all' is not defined

In [24]:
len(period4_records)

936

In [24]:
len(period1_records)

818

Выделяем коллокации по частотности и по t-score

In [25]:
def preprocessed_text(some_text):
    some_text = re.sub(r'\[|\]|([0-9]|\.\.\.|\\\\\—)|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist and morph.parse(word)[0].normal_form not in 
                       extended_punctuation]
    return ' '.join(lemmatized_text)

In [32]:
def get_keywords_bigrams_tf_idf(texts, num_keywords):
    """
    берет список не лемматизированных текстов в виде строк
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова и биграммы, 
    отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer(stop_words=stoplist, ngram_range=(1, 2))
    texts_as_tfidf_vectors = make_tf_idf.fit_transform(preprocessed_text(text) for text in texts)
    id2word = {i : word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0, : -(num_keywords + 1) : -1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [21]:
records5_all_flat = flatten_list(records5_all)
records4_all_flat = flatten_list(records4_all)
records3_all_flat = flatten_list(records3_all)
records2_all_flat = flatten_list(records2_all)
records1_all_flat = flatten_list(records1_all)

In [11]:
with open('period1_records_flat.txt', 'w') as p1:
    for item in records1_all_flat:
        p1.write("%s\n" % item)
with open('period2_records_flat.txt', 'w') as p2:
    for item in records2_all_flat:
        p2.write("%s\n" % item)
with open('period3_records_flat.txt', 'w') as p3:
    for item in records3_all_flat:
        p3.write("%s\n" % item)
with open('period4_records_flat.txt', 'w') as p4:
    for item in records4_all_flat:
        p4.write("%s\n" % item)
with open('period5_records_flat.txt', 'w') as p5:
    for item in records5_all_flat:
        p5.write("%s\n" % item)

In [26]:
period1_records_flat = flatten_list(period1_records)
period2_records_flat = flatten_list(period2_records)

In [27]:
period3_records_flat = flatten_list(period3_records)
period4_records_flat = flatten_list(period4_records)
period5_records_flat = flatten_list(period5_records)

In [32]:
#for text_keywords in get_keywords_bigrams_tf_idf(period1_records_flat, 10):
#    print(text_keywords)

In [73]:
period_list = []
for i, diaries in enumerate(period):
    period_list.append(period[i].text)

In [77]:
period_flat = flatten_list(period_list)

In [ ]:
period_preprocessed = preprocessed_text(period_flat)

In [71]:
period

[ #178009 "Когда родился, не..." @851 [1799-9-16] ,
  #466857 "Первое дорожное неудовольствие,..." @2100 [1802-2-1] ,
  #466858 "Рига [...] Рига..." @2100 [1802-2-4] ,
  ... ,
  #395794 "Утро. Вчера вечером..." @2378 [2017-12-22] ]

In [30]:
period1_preprocessed_flat = preprocessed_text(period1_records_flat)

In [28]:
period1_preprocessed_flat = preprocessed_text(period1_records_flat)
period2_preprocessed_flat = preprocessed_text(period2_records_flat)
period3_preprocessed_flat = preprocessed_text(period3_records_flat)
period4_preprocessed_flat = preprocessed_text(period4_records_flat)
period5_preprocessed_flat = preprocessed_text(period5_records_flat)

In [ ]:
records2_all_flat_prep = preprocessed_text(records2_all_flat)

In [ ]:
records5_all_flat_prep = preprocessed_text(records5_all_flat)
records4_all_flat_prep = preprocessed_text(records4_all_flat)
records3_all_flat_prep = preprocessed_text(records3_all_flat)
records2_all_flat_prep = preprocessed_text(records2_all_flat)
records1_all_flat_prep = preprocessed_text(records1_all_flat)

In [ ]:
type(records5_all_flat_prep)

In [29]:
def collocater(text, number):
    '''функция, которая выдаёт самые частотные коллокации по убыванию'''
    tokens = nltk.wordpunct_tokenize(text)
    tokens_without_punkt = [token for token in tokens if token not in extended_punctuation and stoplist]
    word_prozhito = nltk.FreqDist(tokens_without_punkt)
    bigram_prozhito = nltk.FreqDist(nltk.bigrams(tokens_without_punkt))
    finder_prozhito = BigramCollocationFinder(word_prozhito, bigram_prozhito)
    finder_prozhito.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stoplist)
    finder_prozhito = sorted(finder_prozhito.ngram_fd.items(), key=lambda t: (-t[1], t[0]))
    finder_prozhito = finder_prozhito[:number]
    return finder_prozhito

In [30]:
def collocater_without_limits(text):
    '''функция, которая выдаёт самые частотные коллокации по убыванию без ограничения по количеству'''
    tokens = nltk.wordpunct_tokenize(text)
    tokens_without_punkt = [token for token in tokens if token not in extended_punctuation and stoplist]
    word_prozhito = nltk.FreqDist(tokens_without_punkt)
    bigram_prozhito = nltk.FreqDist(nltk.bigrams(tokens_without_punkt))
    finder_prozhito = BigramCollocationFinder(word_prozhito, bigram_prozhito)
    finder_prozhito.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stoplist)
    finder_prozhito = sorted(finder_prozhito.ngram_fd.items(), key=lambda t: (-t[1], t[0]))
    return finder_prozhito

In [278]:
collocations1 = collocater(period1_preprocessed_flat, 300)

In [279]:
collocations2 = collocater(period2_preprocessed_flat, 300)
collocations3 = collocater(period3_preprocessed_flat, 300)
collocations4 = collocater(period4_preprocessed_flat, 300)
collocations5 = collocater(period5_preprocessed_flat, 300)

In [31]:
collocations1_2 = collocater_without_limits(period1_preprocessed_flat)

In [78]:
print('{0:.10f}'.format(12 / collocations1_overall))

0.0000603494


In [36]:
collocations1_overall = len(collocations1_2)

In [41]:
print(collocations1_overall)
print(collocations2_overall)
print(collocations3_overall)
print(collocations4_overall)
print(collocations5_overall)

198842
458423
261210
180495
194800


In [32]:
collocations2_2 = collocater_without_limits(period2_preprocessed_flat)
collocations2_overall = len(collocations2_2)
collocations3_2 = collocater_without_limits(period3_preprocessed_flat)
collocations3_overall = len(collocations3_2)
collocations4_2 = collocater_without_limits(period4_preprocessed_flat)
collocations4_overall = len(collocations4_2)
collocations5_2 = collocater_without_limits(period5_preprocessed_flat)
collocations5_overall = len(collocations5_2)

In [90]:
for item in collocations3_2:
    for element in item:
        #if element  == ('сердце', 'биться') or element  == ('биться', 'сердце') or element  == ('говорить', 'любовь') or element  == ('взять', 'рука'):
        if element  == ('любимый', 'девушка'):
            print(item)

(('любимый', 'девушка'), 10)


In [92]:
print('{0:.10f}'.format((10) / collocations3_overall))

0.0000382834


In [280]:
collocations1_df = pd.DataFrame(collocations1, columns=['collocation', 'frequency'])

In [281]:
collocations1_df.loc[(collocations1_df['collocation'] == '(..., ...)')]

,collocation,frequency


In [283]:
collocations1_df = collocations1_df.assign(period = 'XIX век')

In [284]:
collocations2_df = pd.DataFrame(collocations2, columns=['collocation', 'frequency'])
collocations2_df = collocations2_df.assign(period = '1899-1938')
collocations3_df = pd.DataFrame(collocations3, columns=['collocation', 'frequency'])
collocations3_df = collocations3_df.assign(period = '1939-1947')
collocations4_df = pd.DataFrame(collocations4, columns=['collocation', 'frequency'])
collocations4_df = collocations4_df.assign(period = '1948-1979')
collocations5_df = pd.DataFrame(collocations5, columns=['collocation', 'frequency'])
collocations5_df = collocations5_df.assign(period = '1980-2017')

In [285]:
df_collocations = pd.concat([collocations1_df, collocations2_df, collocations3_df, collocations4_df, collocations5_df], ignore_index=True)

In [286]:
pip = df_collocations.pivot(index = 'collocation', columns='period', values='frequency')

In [287]:
pip['XIX век'] = pip['XIX век'] / collocations1_overall
pip['1899-1938'] = pip['1899-1938'] / collocations2_overall
pip['1939-1947'] = pip['1939-1947'] / collocations3_overall
pip['1948-1979'] = pip['1948-1979'] / collocations4_overall
pip['1980-2017'] = pip['1980-2017'] / collocations5_overall

In [288]:
pip

period,1899-1938,1939-1947,1948-1979,1980-2017,XIX век
collocation,,,,,
"(абрам, яковлевич)",NaN,NaN,0.000044,NaN,NaN
"(автор, книга)",NaN,NaN,NaN,0.000036,NaN
"(авторский, право)",NaN,NaN,NaN,0.000036,NaN
"(академия, наука)",NaN,0.000092,0.000061,NaN,NaN
"(актовый, зал)",NaN,NaN,0.000033,NaN,NaN
...,...,...,...,...,...
"(шао, цзя)",NaN,0.000084,NaN,NaN,NaN
"(эмма, лида)",NaN,0.000050,NaN,NaN,NaN
"(юрий, гагарин)",NaN,NaN,0.000039,NaN,NaN


In [289]:
import os  
os.makedirs('/Users/victoriavrbv/Documents/Diploma', exist_ok=True)  
pip.to_csv('/Users/victoriavrbv/Documents/Diploma/pip.csv')  

In [114]:
collocation_list = []
for item in period1_preprocessed:
    collocation_list.append(collocater(item, 100))

In [116]:
colloc_for_years_prozhito_100 = {}
for item in period1_preprocessed:
    colloc_for_years_prozhito_100[key] = collocater(item, 100)

In [117]:
colloc_for_years_prozhito_100 

{('„', 'Алые'): [(('александр', 'михаилович'), 1),
  (('безусловно', 'решить'), 1),
  (('бесчестный', 'александр'), 1),
  (('боль', 'глубокий'), 1),
  (('больной', 'вчера'), 1),
  (('больший', 'реальность'), 1),
  (('вновь', 'страшно'), 1),
  (('водка', 'сильно'), 1),
  (('волноваться', 'подождать'), 1),
  (('вопрос', 'подходить'), 1),
  (('вчера', 'выпить'), 1),
  (('вчера', 'земляческий'), 1),
  (('выпить', 'водка'), 1),
  (('глубокий', 'спокойствие'), 1),
  (('говорить', 'вчера'), 1),
  (('горячо', 'любимый'), 1),
  (('господин', '...'), 1),
  (('готовиться', 'дурной'), 1),
  (('грустно', 'счастливый'), 1),
  (('дело', 'избавить'), 1),
  (('должный', 'сбыться'), 1),
  (('дом', 'полный'), 1),
  (('достаточно', 'фраза'), 1),
  (('думать', 'волноваться'), 1),
  (('думать', 'самоубийство'), 1),
  (('думать', 'удивиться'), 1),
  (('дурной', 'прийти'), 1),
  (('ждать', 'письмо'), 1),
  (('замирать', 'готовиться'), 1),
  (('здравый', 'спокойно'), 1),
  (('земляк', 'думать'), 1),
  (('земля

In [125]:
import statistics
log = finder.score_ngrams(bigram_measures.likelihood_ratio)

In [308]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigrams_list = []
tokens = nltk.wordpunct_tokenize(period1_preprocessed_flat)
tokens = [token for token in tokens if token not in extended_punctuation and stoplist2]
finder = BigramCollocationFinder.from_words(tokens)
scored_tm = finder.score_ngrams(bigram_measures.student_t)
#bigrams_list.append(sorted(bigram for bigram, score in scored_tm))
scored_tm = sorted(scored_tm, key=lambda t: (-t[1], t[0]))
scored_tm = scored_tm[:100]
#scored_tm = finder.score_ngrams(bigram_measures.student_t)
#scored_tm = sorted(scored_tm.ngram_fd.items(), key=lambda t: (-t[1], t[0]))
#scored_tm = scored_tm[:number]

In [311]:
def collocater_tscore(text, number):
    tokens = nltk.wordpunct_tokenize(text)
    tokens_without_punkt = [token for token in tokens if token not in extended_punctuation and stoplist]
    finder = BigramCollocationFinder.from_words(tokens_without_punkt)
    scored_tm = finder.score_ngrams(bigram_measures.student_t)
    scored_tm = sorted(scored_tm, key=lambda t: (-t[1], t[0]))
    scored_tm = scored_tm[:number]
    return scored_tm

In [312]:
collocations1_tscore = collocater_tscore(period1_preprocessed_flat, 100)
collocations2_tscore = collocater_tscore(period2_preprocessed_flat, 100)
collocations3_tscore = collocater_tscore(period3_preprocessed_flat, 100)
collocations4_tscore = collocater_tscore(period4_preprocessed_flat, 100)
collocations5_tscore = collocater_tscore(period5_preprocessed_flat, 100)

In [316]:
collocations1_df_tscore = pd.DataFrame(collocations1_tscore, columns=['collocation', 'frequency'])
collocations1_df_tscore = collocations1_df_tscore.assign(period = 'Период 1')

In [317]:
collocations2_df_tscore = pd.DataFrame(collocations2_tscore, columns=['collocation', 'frequency'])
collocations2_df_tscore = collocations2_df_tscore.assign(period = 'Период 2')
collocations3_df_tscore = pd.DataFrame(collocations3_tscore, columns=['collocation', 'frequency'])
collocations3_df_tscore = collocations3_df_tscore.assign(period = 'Период 3')
collocations4_df_tscore = pd.DataFrame(collocations4_tscore, columns=['collocation', 'frequency'])
collocations4_df_tscore = collocations4_df_tscore.assign(period = 'Период 4')
collocations5_df_tscore = pd.DataFrame(collocations5_tscore, columns=['collocation', 'frequency'])
collocations5_df_tscore = collocations5_df_tscore.assign(period = 'Период 5')

In [319]:
df_collocations_tscore = pd.concat([collocations1_df_tscore, collocations2_df_tscore, collocations3_df_tscore, collocations4_df_tscore, collocations5_df_tscore], ignore_index=True)

In [320]:
pip2 = df_collocations_tscore.pivot(index = 'collocation', columns='period', values='collocation')

In [321]:
pip2

period,Период 1,Период 2,Период 3,Период 4,Период 5
collocation,,,,,
"(.\, ть)",NaN,NaN,NaN,NaN,"(.\, ть)"
"(\-, го)",NaN,"(\-, го)","(\-, го)",NaN,NaN
"(\—, \—)","(\—, \—)","(\—, \—)","(\—, \—)","(\—, \—)","(\—, \—)"
"(\—, говорить)","(\—, говорить)","(\—, говорить)",NaN,NaN,NaN
"(\—, знать)","(\—, знать)","(\—, знать)","(\—, знать)","(\—, знать)",NaN
...,...,...,...,...,...
"(юрий, любимов)",NaN,NaN,NaN,NaN,"(юрий, любимов)"
"(я, думать)",NaN,NaN,NaN,NaN,"(я, думать)"
"(я, любить)",NaN,NaN,NaN,"(я, любить)","(я, любить)"


In [322]:
os.makedirs('/Users/victoriavrbv/Documents/Diploma', exist_ok=True)  
pip2.to_csv('/Users/victoriavrbv/Documents/Diploma/pip2.csv')  

In [155]:
#декомпозирую функцию, но решила не использовать
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigrams_list = []
tokens = nltk.wordpunct_tokenize(text)
tokens = [token for token in tokens if token not in extended_punctuation and stoplist2]
finder = BigramCollocationFinder.from_words(tokens)
scored_tm = finder.score_ngrams(bigram_measures.student_t)
bigrams_list.append(sorted(bigram for bigram, score in scored_tm))
#bigrams_list_flat = flatten_list(bigrams_list)

In [44]:
#упрощённая функция, но решила не использовать
def collocater_simple(texts):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigrams_list = []
    for text in texts:
        tokens = nltk.wordpunct_tokenize(text)
        tokens = [token for token in tokens if token not in extended_punctuation and stoplist2]
        finder = BigramCollocationFinder.from_words(tokens)
        scored = finder.score_ngrams(bigram_measures.raw_freq)
        bigrams_list.append(sorted(bigram for bigram, score in scored))
    bigrams_list_flat = flatten_list(bigrams_list)
    colloc_freq = Counter(bigrams_list_flat).most_common()
    return colloc_freq

In [79]:
#collocater_simple(period1_records_flat)

In [53]:
#Counter(bigrams_list_flat).most_common()

In [64]:
colloc_freq = Counter(bigrams_list_flat) 

In [75]:
newdict_5 = {}
for key, value in colloc_freq.items():
    if value > 6 and value < 50:
        newdict_5[key] = value

In [56]:
#newdict_1
#newdict_2

In [57]:
common_list = []
for key1 in newdict_1.keys():
    for key2 in newdict_2.keys():
        if key1 == key2:
            common_list.append(key1)       

In [67]:
len(common_list2)

12395

In [167]:
#считаем количество токенов в периоде
period5_tokenized = []
for item in period5_records:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            period5_tokenized.append(token)
#len(period1_tokenized)

In [168]:
len(period5_tokenized)

458551

In [147]:
#period1_tokenized

In [33]:
with open('period1_records_flat.txt', 'w') as p1:
    for item in period1_records_flat:
        p1.write("%s\n" % item)
with open('period2_records_flat.txt', 'w') as p1:
    for item in period2_records_flat:
        p1.write("%s\n" % item)
with open('period3_records_flat.txt', 'w') as p1:
    for item in period3_records_flat:
        p1.write("%s\n" % item)
with open('period4_records_flat.txt', 'w') as p1:
    for item in period4_records_flat:
        p1.write("%s\n" % item)
with open('period5_records_flat.txt', 'w') as p1:
    for item in period5_records_flat:
        p1.write("%s\n" % item)

In [708]:
#выгружаем записи в txt
with open('period1_records_flat.txt', 'w') as p1:
    for item in period1_records_flat:
        p1.write("%s\n" % item)
with open('period3_records.txt', 'w') as p3:
    for item in period3_records:
        p3.write("%s\n" % item)
with open('period4_records.txt', 'w') as p4:
    for item in period4_records:
        p4.write("%s\n" % item)

In [108]:
with open('period2_records.txt', 'w') as p2:
    for item in period2_records:
        p2.write("%s\n" % item)
with open('period5_records.txt', 'w') as p5:
    for item in period5_records:
        p5.write("%s\n" % item)

In [317]:
#meta_info_cleaned[(meta_info_cleaned['Дата'] > '2013-1-1') & (meta_info_cleaned['Дата'] < '2017-6-8')]
#meta_info_cleaned[(meta_info_cleaned['Дата']>datetime.date(2016, 1, 1)) & (meta_info_cleaned['Дата']<datetime.date(2017, 6, 8))]

In [28]:
#отбираем имена, чтобы разделить по гендеру
names = set()
for item in meta_info_cleaned["Автор"]:
    name = re.search("#\d{1,5}\s(?P<name>[А-ЯЁ][а-яё]+)", str(item))
    if name:
        names.add(name.group ("name"))

In [29]:
female = []
male = []
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне"]
ambig_male = ["Микола", "Илья", "Дзига", "Никита"] 
for name in names:
    if re.search("(а|я|и)$", name) and name not in ambig_male:
        female.append(name)
    elif name in ambig_fem:
        female.append(name)
    else:
         male.append(name)

In [30]:
female_regex = re.compile(""+"|".join(female))
#regex = re.compile("^("+"|".join(item_list)+")\.")
male_regex = re.compile(""+"|".join(male))

In [31]:
#создаем новый столбец с гендером
gender = []
for item in meta_info_cleaned["Автор"]:
    fem_match = re.search(female_regex, str(item))
    male_match = re.search(male_regex, str(item))
    if fem_match:
        gender.append("Женский")
    elif male_match:
        gender.append("Мужской")
    else:
        gender.append("")
meta_info_cleaned["Гендер"] = gender

/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_2000/686961531.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned["Гендер"] = gender


In [32]:
meta_info_cleaned

,ID,Автор,ID дневника,Дата,Гендер
0,140292,#507 Степан Петрович Жихарев,515,1805-07-20,Мужской
1,444870,#719 Николай Дмитриевич Дурново,729,1812-01-22,Мужской
2,178659,#763 Анна Алексеевна Андро,773,1828-10-06,Женский
3,171755,#706 Алексей Николаевич Вульф,716,1831-02-10,Мужской
4,442455,#790 Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский
...,...,...,...,...,...
7136,435616,#1949 Юрий Михайлович Кублановский,1959,2010-03-14,Мужской
7137,394573,#2338 Ирина Львовна Шолохова,2348,2011-01-03,Женский
7139,394581,#2338 Ирина Львовна Шолохова,2348,2012-12-02,Женский
7140,394582,#2338 Ирина Львовна Шолохова,2348,2012-12-29,Женский


In [44]:
#приводим метаданные к единому формату
ids = []
for item in meta_info_cleaned["Автор"]:
    number = re.search(r"\d{1,5}", str(item))
    if number:
        ids.append(number.group())
    else: 
        ids.append(0)
        
p = re.compile(r"#\d{1,5}\s")
meta_info_cleaned["Автор"] = [p.sub('', str(x)) for x in meta_info_cleaned["Автор"]]
        
meta_info_cleaned["ID автора"] = ids   
meta_info_cleaned

/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_50665/2717494215.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned["Автор"] = [p.sub('', str(x)) for x in meta_info_cleaned["Автор"]]
/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_50665/2717494215.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned["ID автора"] = ids


,ID,Автор,ID дневника,Дата,Гендер,ID автора
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790
...,...,...,...,...,...,...
7136,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949
7137,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338
7139,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338
7140,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338


In [45]:
meta_info_cleaned['ID автора']= meta_info_cleaned['ID автора'].astype(int)

/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_50665/106085110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned['ID автора']= meta_info_cleaned['ID автора'].astype(int)


In [46]:
#функция для колонки с количеством записей в дневнике
def record_number(row):
    #for item in tqdm(row["ID дневника"]):
    diary_number = re.compile("@"+str(row["ID дневника"])+"\s")
    for line in dw.diaries:
        if re.search(diary_number, str(line)):
            number = len(line.notes)
            return number
meta_info_cleaned['Количество записей'] = meta_info_cleaned.apply(record_number, axis=1)

KeyboardInterrupt: 

In [33]:
#создаём колонку с периодом записи
period_column = []
for item in tqdm(meta_info_cleaned['Дата']):
    #item = datetime.strptime(item, "%Y-%m-%d")
    if item > datetime.strptime('1798-12-31', "%Y-%m-%d") and item <= datetime.strptime('1898-12-31', "%Y-%m-%d"):
        period_column.append("Период 1")
    elif item > datetime.strptime('1898-12-31', "%Y-%m-%d") and item <= datetime.strptime('1938-12-31', "%Y-%m-%d"):
        period_column.append("Период 2")
    elif item > datetime.strptime('1938-12-31', "%Y-%m-%d") and item <= datetime.strptime('1947-12-31', "%Y-%m-%d"):
        period_column.append("Период 3")
    elif item > datetime.strptime('1947-12-31', "%Y-%m-%d") and item <= datetime.strptime('1979-12-31', "%Y-%m-%d"):
        period_column.append("Период 4")
    elif item > datetime.strptime('1979-12-31', "%Y-%m-%d") and item <= datetime.strptime('2017-12-31', "%Y-%m-%d"):
        period_column.append("Период 5")
    else:
        period_column.append("")

  0%|          | 0/6283 [00:00<?, ?it/s]

In [34]:
#meta_info_cleaned["Количество записей"] = numbers 
meta_info_cleaned["Период"] = period_column

/var/folders/3s/by9v925x5xd0hsnplvbq6j040000gp/T/ipykernel_2000/1912562062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_info_cleaned["Период"] = period_column


In [35]:
meta_info_cleaned

,ID,Автор,ID дневника,Дата,Гендер,Период
0,140292,#507 Степан Петрович Жихарев,515,1805-07-20,Мужской,Период 1
1,444870,#719 Николай Дмитриевич Дурново,729,1812-01-22,Мужской,Период 1
2,178659,#763 Анна Алексеевна Андро,773,1828-10-06,Женский,Период 1
3,171755,#706 Алексей Николаевич Вульф,716,1831-02-10,Мужской,Период 1
4,442455,#790 Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,Период 1
...,...,...,...,...,...,...
7136,435616,#1949 Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,Период 5
7137,394573,#2338 Ирина Львовна Шолохова,2348,2011-01-03,Женский,Период 5
7139,394581,#2338 Ирина Львовна Шолохова,2348,2012-12-02,Женский,Период 5
7140,394582,#2338 Ирина Львовна Шолохова,2348,2012-12-29,Женский,Период 5


In [148]:
meta_info_cleaned.to_csv("prozhito_meta.csv", index=False)

In [40]:
#объединяем датафреймы
full_meta_data = pd.merge(meta_info_cleaned, authors_data, left_on='ID автора', right_on='id', how='left').drop(['id', 'first_name', 'family_name', 'patronymic', 'gender', 'from_natasha', 'from_tags'], axis=1)

In [41]:
full_meta_data

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период,birth_date,death_date,edition,additional_info,info,nickname,wiki
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507,Период 1,1787-02-28,1860-09-12,**Издание:** [* Жихарев С.П.* Записки современ...,NaN,"Писатель, драматург, переводчик, сенатор, теат...",NaN,https://ru.wikipedia.org/wiki/%D0%96%D0%B8%D1%...
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719,Период 1,1725-01-01,1816-12-31,**Издание:** [1812 год... Военные дневники / С...,NaN,"Генерал-аншеф, сенатор.",NaN,"https://ru.wikipedia.org/wiki/Дурново,_Николай..."
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763,Период 1,1808-08-23,1888-12-27,**Издание:** \n* *Оленина А.А. *Дневник. Воспо...,**Критика комментариев к изданию 1999 года: **...,Возлюбленная А.С. Пушкина.,Оленина,"https://ru.wikipedia.org/wiki/Оленина,_Анна_Ал..."
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706,Период 1,1805-12-28,1881-04-29,**Издания:** \n* *А. Н. Вульф*. Дневники : Люб...,NaN,Мемуарист. Близкий друг [А. С. Пушкина](http:/...,NaN,"https://ru.wikipedia.org/wiki/Вульф,_Алексей_Н..."
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790,Период 1,1804-10-26,1863-04-22,**Издания:**\n\n* *Kauchtschwili N.* Il diario...,* *Гиллельсон М. И.* Пушкин в итальянском изда...,Внучка [М. И. Кутузова](http://prozhito.org/pe...,Долли,"https://ru.wikipedia.org/wiki/Фикельмон,_Дарья..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949,Период 5,1947-04-30,NaN,**Издание:** *Кублановский Ю.* Год за год // Н...,NaN,"Поэт и эссеист, публицист, критик, искусствовед.",NaN,"https://ru.wikipedia.org/wiki/Кублановский,_Юр..."
6279,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN
6280,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN
6281,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN


In [42]:
full_meta_data['birth_date'] = pd.to_datetime(full_meta_data['birth_date'], format='%Y-%m-%d')
full_meta_data['death_date'] = pd.to_datetime(full_meta_data['death_date'], format='%Y-%m-%d', errors = 'coerce')
full_meta_data['Дата'] = pd.to_datetime(full_meta_data['Дата'], format='(%Y, %m, %d)')

In [43]:
full_meta_data['date_year'] = full_meta_data.Дата.apply(lambda x: x.year)
full_meta_data['birth_date_year'] = full_meta_data.birth_date.apply(lambda x: x.year)
full_meta_data['age'] = full_meta_data['date_year'] - full_meta_data['birth_date_year']

In [44]:
full_meta_data.drop(['date_year', 'birth_date_year'], axis=1)

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период,birth_date,death_date,edition,additional_info,info,nickname,wiki,age
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507,Период 1,1787-02-28,1860-09-12,**Издание:** [* Жихарев С.П.* Записки современ...,NaN,"Писатель, драматург, переводчик, сенатор, теат...",NaN,https://ru.wikipedia.org/wiki/%D0%96%D0%B8%D1%...,18.0
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719,Период 1,1725-01-01,1816-12-31,**Издание:** [1812 год... Военные дневники / С...,NaN,"Генерал-аншеф, сенатор.",NaN,"https://ru.wikipedia.org/wiki/Дурново,_Николай...",87.0
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763,Период 1,1808-08-23,1888-12-27,**Издание:** \n* *Оленина А.А. *Дневник. Воспо...,**Критика комментариев к изданию 1999 года: **...,Возлюбленная А.С. Пушкина.,Оленина,"https://ru.wikipedia.org/wiki/Оленина,_Анна_Ал...",20.0
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706,Период 1,1805-12-28,1881-04-29,**Издания:** \n* *А. Н. Вульф*. Дневники : Люб...,NaN,Мемуарист. Близкий друг [А. С. Пушкина](http:/...,NaN,"https://ru.wikipedia.org/wiki/Вульф,_Алексей_Н...",26.0
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790,Период 1,1804-10-26,1863-04-22,**Издания:**\n\n* *Kauchtschwili N.* Il diario...,* *Гиллельсон М. И.* Пушкин в итальянском изда...,Внучка [М. И. Кутузова](http://prozhito.org/pe...,Долли,"https://ru.wikipedia.org/wiki/Фикельмон,_Дарья...",27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949,Период 5,1947-04-30,NaT,**Издание:** *Кублановский Ю.* Год за год // Н...,NaN,"Поэт и эссеист, публицист, критик, искусствовед.",NaN,"https://ru.wikipedia.org/wiki/Кублановский,_Юр...",63.0
6279,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,40.0
6280,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,41.0
6281,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,41.0


In [44]:
#считаем возрастные группы
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0
for item in full_meta_data["age"]:
    if item <= 16:
        counter1 += 1
for item in full_meta_data["age"]:
    if item > 16 and item <= 26: 
        counter2 += 1
for item in full_meta_data["age"]:
    if item > 26 and item <= 41: 
        counter3 += 1
for item in full_meta_data["age"]:
    if item > 41 and item <= 60: 
        counter4 += 1
for item in full_meta_data["age"]:
    if item > 60: 
        counter5 += 1
print(counter1, counter2, counter3, counter4, counter5)

440 1705 1486 1283 894


In [407]:
#meta_info_cleaned[meta_info_cleaned['Автор'].str.contains("#78 Евгений Львович Шварц")]

In [45]:
#Объединяем авторов в один датафрейм, чтобы посчитать среднее эмоциональное

authors4_corpus = meta_4.Автор.value_counts()
authors4_corpus = pd.DataFrame(authors4_corpus)
authors4_corpus = authors4_corpus.reset_index()
authors4_corpus.columns = ["Author", "2"]

In [47]:
len(authors4_corpus["Author"])

138

In [48]:
authors4_full = pd.merge(authors_4, authors4_corpus, on="Author")

NameError: name 'authors_4' is not defined

In [494]:
new_column = authors4_full["2"] / authors4_full["1"]
authors4_full["3"] = new_column
authors4_full["3"].mean()

0.054407282287850876

In [500]:
#То же самое с 3 периодом
authors3_corpus = meta_3.Автор.value_counts()
authors3_corpus = pd.DataFrame(authors3_corpus)
authors3_corpus = authors3_corpus.reset_index()
authors3_corpus.columns = ["Author", "2"]

In [501]:
authors3_full = pd.merge(authors_3, authors3_corpus, on="Author")

In [502]:
new_column = authors3_full["2"] / authors3_full["1"]
authors3_full["3"] = new_column
authors3_full["3"].mean()

0.05215995940294942

In [512]:
#Второй период
authors2_corpus = meta_2.Автор.value_counts()
authors2_corpus = pd.DataFrame(authors2_corpus)
authors2_corpus = authors2_corpus.reset_index()
authors2_corpus.columns = ["Author", "2"]

In [513]:
authors2_full = pd.merge(authors_2, authors2_corpus, on="Author")
new_column = authors2_full["2"] / authors2_full["1"]
authors3_full["3"] = new_column
authors3_full["3"].mean()

0.042479003633874815

In [514]:
#Первый период
authors1_corpus = meta_1.Автор.value_counts()
authors1_corpus = pd.DataFrame(authors1_corpus)
authors1_corpus = authors1_corpus.reset_index()
authors1_corpus.columns = ["Author", "2"]

In [515]:
authors1_full = pd.merge(authors_1, authors1_corpus, on="Author")
new_column = authors1_full["2"] / authors1_full["1"]
authors1_full["3"] = new_column
authors1_full["3"].mean()

0.06629841037155669

In [527]:
#Пятый период
authors5_corpus = meta_5.Автор.value_counts()
authors5_corpus = pd.DataFrame(authors5_corpus)
authors5_corpus = authors5_corpus.reset_index()
authors5_corpus.columns = ["Author", "2"]

In [528]:
authors5_full = pd.merge(authors_5, authors5_corpus, on="Author")
new_column = authors5_full["2"] / authors5_full["1"]
authors5_full["3"] = new_column
authors5_full["3"].mean()

0.04132390911505409

In [39]:
#meta_4["Автор"].count()
#pd.set_option("display.max_rows", 20, "display.max_columns", None)
pd.set_option('display.max_rows', None)
meta_4.Автор.value_counts()

#981 Галина Г. Ларская                                           73
#78 Евгений Львович Шварц                                        46
#565 Юлия Львовна Нельская-Сидур                                 34
#1533 Нина Сергеевна Покровская                                  28
#2477 Анхель  Гутьеррес                                          25
#1285 Лидия Владимировна Барлас                                  22
#1023 Яков Семёнович Друскин                                     21
#959 Зинаида Степановна Лелянова                                 19
#48 Николай Дмитриевич Мордвинов                                 19
#215 Корней Иванович Чуковский                                   17
#323 Георгий Анатольевич Елин                                    16
#229 Александр Константинович Гладков                            16
#1385 Эльвира Григорьевна Филипович                              16
#257 Николай Николаевич Козаков                                  16
#102 Валерий Сергеевич Золотухин                

In [91]:
#считаем, сколько женщин и мужчин в корпусе
#meta_info_cleaned.groupby('Автор')('Гендер').value_counts()
meta_info_cleaned.groupby('Гендер')['Автор'].nunique()

Гендер
            12
Женский    243
Мужской    444
Name: Автор, dtype: int64

In [49]:
#выбираем ID записей женщин и мужчин
female_ids = []
male_ids = []
for item in meta_info_cleaned[meta_info_cleaned['Гендер'] == 'Женский']["ID"]:
    female_ids.append(item)
for item in meta_info_cleaned[meta_info_cleaned['Гендер'].str.match('Мужской')]["ID"]:
    male_ids.append(item)

In [50]:
def gender_period_ids(gender, period):
    '''Функция принимает на вход гендер и период и создаёт список с идентификаторами записей'''
    gender_period_ids_list = meta_info_cleaned[(meta_info_cleaned['Гендер'] == gender) & (meta_info_cleaned['Период'] == period)]["ID"].tolist()
    return gender_period_ids_list

In [51]:
female_ids1 = gender_period_ids('Женский', 'Период 1')
female_ids2 = gender_period_ids('Женский', 'Период 2')
female_ids3 = gender_period_ids('Женский', 'Период 3')
female_ids4 = gender_period_ids('Женский', 'Период 4')
female_ids5 = gender_period_ids('Женский', 'Период 5')
male_ids1 = gender_period_ids('Мужской', 'Период 1')
male_ids2 = gender_period_ids('Мужской', 'Период 2')
male_ids3 = gender_period_ids('Мужской', 'Период 3')
male_ids4 = gender_period_ids('Мужской', 'Период 4')
male_ids5 = gender_period_ids('Мужской', 'Период 5')

In [49]:
def gender_period_records(ids_list):
    '''Функция принимает на вход список c идентификаторами и создаёт список с записями'''
    records_list = []
    for item in diaries_dic.items():
        for x in ids_list:
            if item[0] == x:
                records_list.append(item[1])
    return records_list

In [52]:
female_records1 = gender_period_records(female_ids1)
female_records2 = gender_period_records(female_ids2)
female_records3 = gender_period_records(female_ids3)
female_records4 = gender_period_records(female_ids4)
female_records5 = gender_period_records(female_ids5)
male_records1 = gender_period_records(male_ids1)
male_records2 = gender_period_records(male_ids2)
male_records3 = gender_period_records(male_ids3)
male_records4 = gender_period_records(male_ids4)
male_records5 = gender_period_records(male_ids5)

In [53]:
female_records1_flat = flatten_list(female_records1)
female_records1_preprocessed_flat = preprocessed_text(female_records1_flat)
female_records2_flat = flatten_list(female_records2)
female_records2_preprocessed_flat = preprocessed_text(female_records2_flat)
female_records3_flat = flatten_list(female_records3)
female_records3_preprocessed_flat = preprocessed_text(female_records3_flat)
female_records4_flat = flatten_list(female_records4)
female_records4_preprocessed_flat = preprocessed_text(female_records4_flat)
female_records5_flat = flatten_list(female_records5)
female_records5_preprocessed_flat = preprocessed_text(female_records5_flat)
male_records1_flat = flatten_list(male_records1)
male_records1_preprocessed_flat = preprocessed_text(male_records1_flat)
male_records2_flat = flatten_list(male_records2)
male_records2_preprocessed_flat = preprocessed_text(male_records2_flat)
male_records3_flat = flatten_list(male_records3)
male_records3_preprocessed_flat = preprocessed_text(male_records3_flat)
male_records4_flat = flatten_list(male_records4)
male_records4_preprocessed_flat = preprocessed_text(male_records4_flat)
male_records5_flat = flatten_list(male_records5)
male_records5_preprocessed_flat = preprocessed_text(male_records5_flat)

In [487]:
collocations_female1 = collocater(female_records1_preprocessed_flat, 200)
collocations_female2 = collocater(female_records2_preprocessed_flat, 200)
collocations_female3 = collocater(female_records3_preprocessed_flat, 200)
collocations_female4 = collocater(female_records4_preprocessed_flat, 200)
collocations_female5 = collocater(female_records5_preprocessed_flat, 200)
collocations_male1 = collocater(male_records1_preprocessed_flat, 200)
collocations_male2 = collocater(male_records2_preprocessed_flat, 200)
collocations_male3 = collocater(male_records3_preprocessed_flat, 200)
collocations_male4 = collocater(male_records4_preprocessed_flat, 200)
collocations_male5 = collocater(male_records5_preprocessed_flat, 200)

In [57]:
collocations_female1_all = collocater_without_limits(female_records1_preprocessed_flat)
collocations_female1_overall = len(collocations_female1_all)
collocations_female2_all = collocater_without_limits(female_records2_preprocessed_flat)
collocations_female2_overall = len(collocations_female2_all)
collocations_female3_all = collocater_without_limits(female_records3_preprocessed_flat)
collocations_female3_overall = len(collocations_female3_all)
collocations_female4_all = collocater_without_limits(female_records4_preprocessed_flat)
collocations_female4_overall = len(collocations_female4_all)
collocations_female5_all = collocater_without_limits(female_records5_preprocessed_flat)
collocations_female5_overall = len(collocations_female5_all)
collocations_male1_all = collocater_without_limits(male_records1_preprocessed_flat)
collocations_male1_overall = len(collocations_male1_all)
collocations_male2_all = collocater_without_limits(male_records2_preprocessed_flat)
collocations_male2_overall = len(collocations_male2_all)
collocations_male3_all = collocater_without_limits(male_records3_preprocessed_flat)
collocations_male3_overall = len(collocations_male3_all)
collocations_male4_all = collocater_without_limits(male_records4_preprocessed_flat)
collocations_male4_overall = len(collocations_male4_all)
collocations_male5_all = collocater_without_limits(male_records5_preprocessed_flat)
collocations_male5_overall = len(collocations_male5_all)

In [127]:
for item in collocations_female2_all:
    for element in item:
        if element  == ('сердце', 'биться') or element  == ('биться', 'сердце'):
            print(item)

(('сердце', 'биться'), 14)
(('биться', 'сердце'), 9)


In [122]:
fratio1 = 6 / collocations_female1_overall
fratio2 = 23 / collocations_female2_overall
fratio3 = 6 / collocations_female3_overall
fratio4 = 2 / collocations_female4_overall
fratio5 = 1 / collocations_female5_overall
mratio1 = 8 / collocations_male1_overall
mratio2 = 11 / collocations_male2_overall
mratio3 = 4 / collocations_male3_overall
mratio4 = 0 / collocations_male4_overall

In [73]:
print('{0:.15f}'.format(5.432387896639766e-05))

0.000054323878966


In [123]:
print('{0:.15f}'.format(fratio1), '{0:.15f}'.format(fratio2), '{0:.15f}'. format(fratio3), '{0:.15f}'.format(fratio4), '{0:.15f}'.format(fratio5), '{0:.15f}'.format(mratio1), '{0:.15f}'.format(mratio2), '{0:.15f}'.format(mratio3), '{0:.15f}'.format(mratio4))

0.000109134562916 0.000138827690692 0.000060667340748 0.000030554876558 0.000010459266387 0.000053709298422 0.000035672359110 0.000023454632876 0.000000000000000


In [489]:
collocations1_female_df = pd.DataFrame(collocations_female1, columns=['collocation', 'frequency'])
collocations1_female_df = collocations1_female_df.assign(period = 'Период 1', gender = 'Женский')
collocations1_female_df['frequency'] = collocations1_female_df['frequency'] / collocations_female1_overall
collocations2_female_df = pd.DataFrame(collocations_female2, columns=['collocation', 'frequency'])
collocations2_female_df = collocations2_female_df.assign(period = 'Период 2', gender = 'Женский')
collocations2_female_df['frequency'] = collocations2_female_df['frequency'] / collocations_female2_overall
collocations3_female_df = pd.DataFrame(collocations_female3, columns=['collocation', 'frequency'])
collocations3_female_df = collocations3_female_df.assign(period = 'Период 3', gender = 'Женский')
collocations3_female_df['frequency'] = collocations3_female_df['frequency'] / collocations_female3_overall
collocations4_female_df = pd.DataFrame(collocations_female4, columns=['collocation', 'frequency'])
collocations4_female_df = collocations4_female_df.assign(period = 'Период 4', gender = 'Женский')
collocations4_female_df['frequency'] = collocations4_female_df['frequency'] / collocations_female4_overall
collocations5_female_df = pd.DataFrame(collocations_female5, columns=['collocation', 'frequency'])
collocations5_female_df = collocations5_female_df.assign(period = 'Период 5', gender = 'Женский')
collocations5_female_df['frequency'] = collocations5_female_df['frequency'] / collocations_female5_overall
collocations1_male_df = pd.DataFrame(collocations_male1, columns=['collocation', 'frequency'])
collocations1_male_df = collocations1_male_df.assign(period = 'Период 1', gender = 'Мужской')
collocations1_male_df['frequency'] = collocations1_male_df['frequency'] / collocations_male1_overall
collocations2_male_df = pd.DataFrame(collocations_male2, columns=['collocation', 'frequency'])
collocations2_male_df = collocations2_male_df.assign(period = 'Период 2', gender = 'Мужской')
collocations2_male_df['frequency'] = collocations2_male_df['frequency'] / collocations_male2_overall
collocations3_male_df = pd.DataFrame(collocations_male3, columns=['collocation', 'frequency'])
collocations3_male_df = collocations3_male_df.assign(period = 'Период 3', gender = 'Мужской')
collocations3_male_df['frequency'] = collocations3_male_df['frequency'] / collocations_male3_overall
collocations4_male_df = pd.DataFrame(collocations_male4, columns=['collocation', 'frequency'])
collocations4_male_df = collocations4_male_df.assign(period = 'Период 4', gender = 'Мужской')
collocations4_male_df['frequency'] = collocations4_male_df['frequency'] / collocations_male4_overall
collocations5_male_df = pd.DataFrame(collocations_male5, columns=['collocation', 'frequency'])
collocations5_male_df = collocations5_male_df.assign(period = 'Период 5', gender = 'Мужской')
collocations5_male_df['frequency'] = collocations5_male_df['frequency'] / collocations_male5_overall

In [490]:
df_collocations_gender_period = pd.concat([collocations1_female_df, collocations2_female_df, collocations3_female_df, collocations4_female_df, collocations5_female_df, collocations1_male_df, collocations2_male_df, collocations3_male_df, collocations4_male_df, collocations5_male_df], ignore_index=True)

In [491]:
pip3 = df_collocations_gender_period.pivot(index = 'collocation', columns=['period', 'gender'], values='frequency')

In [492]:
pip3

period,Период 1,Период 2,Период 3,Период 4,Период 5,Период 1,Период 2,Период 3,Период 4,Период 5
gender,Женский,Женский,Женский,Женский,Женский,Мужской,Мужской,Мужской,Мужской,Мужской
collocation,,,,,,,,,,
"(абрам, яковлевич)",NaN,NaN,NaN,0.000122,NaN,NaN,NaN,NaN,NaN,NaN
"(абсолютный, субъективность)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000042,NaN
"(автор, книга)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000059
"(авторский, право)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000069
"(академия, наука)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000135,0.000067,NaN
...,...,...,...,...,...,...,...,...,...,...
"(юлия, павлович)",0.000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(юрий, гагарин)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000059,NaN


In [493]:
os.makedirs('/Users/victoriavrbv/Documents/Diploma', exist_ok=True)  
pip3.to_csv('/Users/victoriavrbv/Documents/Diploma/pip3.csv')  

In [368]:
collocations_tscore_female1 = collocater_tscore(female_records1_preprocessed_flat, 100)
collocations_tscore_female2 = collocater_tscore(female_records2_preprocessed_flat, 100)
collocations_tscore_female3 = collocater_tscore(female_records3_preprocessed_flat, 100)
collocations_tscore_female4 = collocater_tscore(female_records4_preprocessed_flat, 100)
collocations_tscore_female5 = collocater_tscore(female_records5_preprocessed_flat, 100)
collocations_tscore_male1 = collocater_tscore(male_records1_preprocessed_flat, 100)
collocations_tscore_male2 = collocater_tscore(male_records2_preprocessed_flat, 100)
collocations_tscore_male3 = collocater_tscore(male_records3_preprocessed_flat, 100)
collocations_tscore_male4 = collocater_tscore(male_records4_preprocessed_flat, 100)
collocations_tscore_male5 = collocater_tscore(male_records5_preprocessed_flat, 100)

In [370]:
collocations1_female_df_tscore = pd.DataFrame(collocations_tscore_female1, columns=['collocation', 'frequency'])
collocations1_female_df_tscore = collocations1_female_df_tscore.assign(period = 'Период 1', gender = 'Женский')
collocations2_female_df_tscore = pd.DataFrame(collocations_tscore_female2, columns=['collocation', 'frequency'])
collocations2_female_df_tscore = collocations2_female_df_tscore.assign(period = 'Период 2', gender = 'Женский')
collocations3_female_df_tscore = pd.DataFrame(collocations_tscore_female3, columns=['collocation', 'frequency'])
collocations3_female_df_tscore = collocations3_female_df_tscore.assign(period = 'Период 3', gender = 'Женский')
collocations4_female_df_tscore = pd.DataFrame(collocations_tscore_female4, columns=['collocation', 'frequency'])
collocations4_female_df_tscore = collocations4_female_df_tscore.assign(period = 'Период 4', gender = 'Женский')
collocations5_female_df_tscore = pd.DataFrame(collocations_tscore_female5, columns=['collocation', 'frequency'])
collocations5_female_df_tscore = collocations5_female_df_tscore.assign(period = 'Период 5', gender = 'Женский')
collocations1_male_df_tscore = pd.DataFrame(collocations_tscore_male1, columns=['collocation', 'frequency'])
collocations1_male_df_tscore = collocations1_male_df_tscore.assign(period = 'Период 1', gender = 'Мужской')
collocations2_male_df_tscore = pd.DataFrame(collocations_tscore_male2, columns=['collocation', 'frequency'])
collocations2_male_df_tscore = collocations2_male_df_tscore.assign(period = 'Период 2', gender = 'Мужской')
collocations3_male_df_tscore = pd.DataFrame(collocations_tscore_male3, columns=['collocation', 'frequency'])
collocations3_male_df_tscore = collocations3_male_df_tscore.assign(period = 'Период 3', gender = 'Мужской')
collocations4_male_df_tscore = pd.DataFrame(collocations_tscore_male4, columns=['collocation', 'frequency'])
collocations4_male_df_tscore = collocations4_male_df_tscore.assign(period = 'Период 4', gender = 'Мужской')
collocations5_male_df_tscore = pd.DataFrame(collocations_tscore_male5, columns=['collocation', 'frequency'])
collocations5_male_df_tscore = collocations5_male_df_tscore.assign(period = 'Период 5', gender = 'Мужской')

In [371]:
df_collocations_gender_period_tscore = pd.concat([collocations1_female_df_tscore, collocations2_female_df_tscore, collocations3_female_df_tscore, collocations4_female_df_tscore, collocations5_female_df_tscore, collocations1_male_df_tscore, collocations2_male_df_tscore, collocations3_male_df_tscore, collocations4_male_df_tscore, collocations5_male_df_tscore], ignore_index=True)

In [372]:
pip4 = df_collocations_gender_period_tscore.pivot(index = 'collocation', columns=['period', 'gender'], values='collocation')

In [373]:
pip4

period,Период 1,Период 2,Период 3,Период 4,Период 5,Период 1,Период 2,Период 3,Период 4,Период 5
gender,Женский,Женский,Женский,Женский,Женский,Мужской,Мужской,Мужской,Мужской,Мужской
collocation,,,,,,,,,,
"('-, го)","('-, го)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(.\, м)",NaN,NaN,NaN,NaN,"(.\, м)",NaN,NaN,NaN,NaN,NaN
"(.\, п)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(.\, п)",NaN,NaN
"(.\, ть)",NaN,NaN,NaN,NaN,"(.\, ть)",NaN,NaN,NaN,NaN,NaN
"(.\, ф)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(.\, ф)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
"(я, картина)",NaN,NaN,"(я, картина)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(я, любить)",NaN,NaN,NaN,NaN,"(я, любить)",NaN,NaN,NaN,"(я, любить)",NaN


In [374]:
os.makedirs('/Users/victoriavrbv/Documents/Diploma', exist_ok=True)  
pip4.to_csv('/Users/victoriavrbv/Documents/Diploma/pip4.csv')  

In [84]:
#выгружаем записи в txt
with open('female_records.txt', 'w') as f:
    for item in female_records:
        f.write("%s\n" % item)

In [85]:
with open('male_records.txt', 'w') as m:
    for item in male_records:
        m.write("%s\n" % item)

In [390]:
#не получилось
fem = []
for item in diaries_dic.items():
    if item[0] == (x for x in female_ids):
        fem.append(item)

In [188]:
#лемматизируем корпус
for key, value in diaries_dic.items():
    diaries_dic[key] = preprocess_text(diaries_dic[key])

In [280]:
#diaries_dic

In [94]:
def flatten_list(_2d_list):
    '''
    берет вложенный список, возвращает плоский
    '''
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [111]:
period1_records_flat = flatten_list(period1_records)

In [455]:
#экспериментальный чанк 
period1_tokenized_records = []
for item in period1_records_flat:
    items = word_tokenize(item.lower())
    period1_tokenized_records.append(items)

In [46]:
all_records = []
for item in diaries_dic.items():
    all_records.append(item[1])

In [47]:
all_records_flat = flatten_list(all_records)

In [48]:
#period1_records_2
periods_lemmatized = []
for item in all_records_flat:
    periods_lemmatized.append(preprocess_text(item))

In [43]:
period1_lemmatized = []
for item in period1_records_flat:
    period1_lemmatized.append(preprocess_text(item))

In [54]:
names_list = ["степанович", "алик", "люся", "ир", "екатерина", "лёня", "люба", "герман", "митя", "давид", "вячеслав", "алла", "галя", "эфроса", "витя", "губенко", "лина", "наталья", "мишка", "валерий", "дима", "лариса", "аня", "ляла", "татьяна", "аксюша", "юзеф", "дмитрий", "алексей", "дмитриевич", "петров", "петя", "андрей", "гриша", "боря", "николаев", "аля", "елена", "сереж", "григориевич", "василий", "сомовый", "павлик", "шура", "наташа", "катя", "александрович", "иван", "юрий", "надя", "владимир", "лида", "виктор", "иванов", "ирина", "валя", "борис", "анович", "мария", "чертков", "сергеевич", "ольга", "лёля", "фёдорович", "григорий", "михаилович", "николаевич", "анна", "пётр", "петрович", "васильевич", "ляля", "иванович", "зина", "володя", "никита", "лена", "коля", "николай", "маруся", "лиза", "олег", "вася", "игорь", "георгий", "львович", "марина", "софья", "андреевич", "александр", "цветаев", "владимирович", "таня", "анна", "василиевич", "петрович", "сергей", "василиевич", "саша", "павлович", "маша", "миша", "константин", "пётр", "павел", "лена", "лев", "серёжа", "оля", "яковлевич", "нина", "шурик", "женя", "соня", "семён", "костя", "миша"]
for item in periods_lemmatized:
    for i in range(len(item)):
        if item[i] in names_list:
            item[i] = "<имя>"
        if re.search("[а-я]\.", str(item[i])):
            item[i] = "<имя>"
    #item = [element for element in item if re.search("^(?!(?:[а-я]\.)$).*$", str(element))]
    #if re.search("[а-я]\.", records_lemmatized[i])

#period1_lemmatized = [item for item in period1_lemmatized if re.search("^(?!(?:[а-я]\.)$).*$", str(item))]

In [659]:
#period1_lemmatized_flat = flatten_list(period1_lemmatized)

In [255]:
#period1_corrected

In [44]:
period1_corrected = [" ".join(x) for x in period1_lemmatized]

In [49]:
def get_topics(model, feature_names, no_top_words):
    '''
    берет модель топик-моделинга, слова и количество топиков, возврашает темы
    '''
    with open('topics_period1_lemm.txt', 'w', encoding='utf-8') as f:
        for topic_idx, topic in enumerate(model.components_):
            print(f"Тема {topic_idx+1}:", file=f)
            topic_words = ", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
            print(topic_words, file=f)

In [50]:
new_stoplist = stoplist2+names_list

In [51]:
def main():
    tfidf_vectorizer = TfidfVectorizer(max_df=0.1, min_df=10, ngram_range=(2, 2), stop_words=new_stoplist)
    tfidf = tfidf_vectorizer.fit_transform(period1_corrected)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    nmf = NMF(n_components=20, random_state=42, alpha=.1, l1_ratio=.5, init='nndsvd', max_iter=100000).fit(tfidf)
    get_topics(nmf, tfidf_feature_names, 10)

In [52]:
if __name__ == '__main__':
    main()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['франц'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [190]:
import gensim

In [259]:
def preprocess(input_text):
    '''функция для предобработки текста и замены имён'''
    ## токенизируем через nltk:
    tokenized_text = word_tokenize (input_text.lower())
    ## убираем пунктуацию и стоп-слова:
    clean_text = [word for word in tokenized_text if word not in stoplist and word not in extended_punctuation]
    ## лемматизируем pymorphy2:
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]
    names_list = ["степанович", "алик", "люся", "ир", "екатерина", "лёня", "люба", "герман", "митя", "давид", "вячеслав", "алла", "галя", "эфроса", "витя", "губенко", "лина", "наталья", "мишка", "валерий", "дима", "лариса", "аня", "ляла", "татьяна", "аксюша", "юзеф", "дмитрий", "алексей", "дмитриевич", "петров", "петя", "андрей", "гриша", "боря", "николаев", "аля", "елена", "сереж", "григориевич", "василий", "сомовый", "павлик", "шура", "наташа", "катя", "александрович", "иван", "юрий", "надя", "владимир", "лида", "виктор", "иванов", "ирина", "валя", "борис", "анович", "мария", "чертков", "сергеевич", "ольга", "лёля", "фёдорович", "григорий", "михаилович", "николаевич", "анна", "пётр", "петрович", "васильевич", "ляля", "иванович", "зина", "володя", "никита", "лена", "коля", "николай", "маруся", "лиза", "олег", "вася", "игорь", "георгий", "львович", "марина", "софья", "андреевич", "александр", "цветаев", "владимирович", "таня", "анна", "василиевич", "петрович", "сергей", "василиевич", "саша", "павлович", "маша", "миша", "константин", "пётр", "павел", "лена", "лев", "серёжа", "оля", "яковлевич", "нина", "шурик", "женя", "соня", "семён", "костя", "миша"]
    for item in lemmatized_text:
        for i in range(len(item)):
            if item[i] in names_list:
                item[i] = "<имя>"
                if re.search("[а-я]\.", str(item[i])):
                    item[i] = "<имя>"
    return lemmatized_text

In [260]:
period2_preprocessed = [preprocess(record) for record in period2_records_flat]

In [234]:
period1_preprocessed_flat = flatten_list(period1_preprocessed)

In [261]:
#чанк для вытаскивания фраз
phrases = gensim.models.Phrases(period2_preprocessed)
p = gensim.models.phrases.Phraser(phrases)
ngrammed_texts = p [period2_preprocessed]

In [262]:
gensim_dictionary_for_TM = gensim.corpora.Dictionary(period2_preprocessed)

In [263]:
gensim_dictionary_for_TM.filter_extremes(no_above=0.1, no_below=20)

In [264]:
gensim_dictionary_for_TM.compactify()

In [265]:
corpus = [gensim_dictionary_for_TM.doc2bow(text) for text in period2_preprocessed]

In [266]:
lda2 = gensim.models.LdaMulticore(corpus, 
                                 num_topics = 20, # число топиков
                                 id2word=gensim_dictionary_for_TM, 
                                 passes=5)

In [267]:
lda2.print_topics()

[(0,
  '0.003*"небо" + 0.003*"собака" + 0.003*"тоска" + 0.003*"лес" + 0.002*"природа" + 0.002*"смеяться" + 0.002*"господь" + 0.002*"тело" + 0.002*"страшный" + 0.002*"вода"'),
 (1,
  '0.005*"личный" + 0.005*"личность" + 0.004*"борьба" + 0.003*"вода" + 0.003*"человеческий" + 0.003*"общество" + 0.003*"искусство" + 0.003*"цель" + 0.003*"сознание" + 0.003*"гора"'),
 (2,
  '0.005*"тётя" + 0.004*"саша" + 0.004*"лошадь" + 0.004*"мужик" + 0.003*"лес" + 0.003*"серёжа" + 0.003*"красный" + 0.003*"сон" + 0.003*"дерево" + 0.003*"дождь"'),
 (3,
  '0.009*"пьеса" + 0.008*"дягилев" + 0.008*"париж" + 0.005*"артист" + 0.005*"сцена" + 0.005*"репетиция" + 0.004*"концерт" + 0.004*"сентябрь" + 0.004*"роль" + 0.004*"операция"'),
 (4,
  '0.051*"л." + 0.044*"н." + 0.021*"андреевич" + 0.020*"софья" + 0.013*"львович" + 0.010*"григориевич" + 0.009*"владимир" + 0.008*"ясный" + 0.007*"александр" + 0.006*"h."'),
 (5,
  '0.006*"26" + 0.005*"соня" + 0.005*"надя" + 0.004*"царь" + 0.004*"27" + 0.004*"надежда" + 0.004*"руб

In [40]:
#запишем записи нужного периода в словарь чтобы не потерять ID
def period_to_dict(list_ids):
    '''Функция собирает в словарь ID и текст записи нужного периода'''
    period_dict = {}
    for key, value in diaries_dic.items():
        for item in list_ids:
            if key == item:
                period_dict[key]= value
    return period_dict

In [54]:
female1_dict = period_to_dict(female_ids1)
female2_dict = period_to_dict(female_ids2)
female3_dict = period_to_dict(female_ids3)
female4_dict = period_to_dict(female_ids4)
female5_dict = period_to_dict(female_ids5)
male1_dict = period_to_dict(male_ids1)
male2_dict = period_to_dict(male_ids2)
male3_dict = period_to_dict(male_ids3)
male4_dict = period_to_dict(male_ids4)
male5_dict = period_to_dict(male_ids5)

In [41]:
period1_dict = period_to_dict(period1_ids)

In [42]:
period2_dict = period_to_dict(period2_ids)
period3_dict = period_to_dict(period3_ids)
period4_dict = period_to_dict(period4_ids)
period5_dict = period_to_dict(period5_ids)

In [56]:
#пополняем данные записей с паттернами
meta_info_cleaned[(meta_info_cleaned['ID'] == 187649)]

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период
609,187649,Юрий Алексеевич Рябов,528,1942-12-11,Мужской,520,Период 3


In [83]:
#ищем записи Галины Ларской из пятого периода
larskaya_ids = meta_info_cleaned[meta_info_cleaned['ID дневника'] == 991]["ID"]

In [89]:
larskaya_records = []
for item in diaries_dic.items():
    for x in larskaya_ids:
        if item[0] == x:
            larskaya_records.append(item[1]) 

In [115]:
larskaya_records_flat = flatten_list(larskaya_records)

In [86]:
more_stops = ['мой', 'один', 'который', 'свой', 'такой', 'иметь', 'должный', 'сделать', 'дать', 'наш', 'самый', 'всякий', 'первый', 'стать', 'к', 'идти', 'какой', 'мочь', '\\nя', 'другой', 'прийти', 'новый', 'какой-то', 'хотеться', 'никто', 'давать', 'взять', 'из-за', 'com', 'id=', '\\n—', '\\n', "''", 'который', "..."]

In [103]:
from string import punctuation
more_punctuation = punctuation + "—»«...–''"

In [107]:
from pymorphy2 import MorphAnalyzer
parser = MorphAnalyzer ()

In [105]:
def keywords_most_frequent_lemma (some_list, num_most_freq):
    '''Функция получает на вход текст и число, выдаёт указанное число частотных слов'''
    for string in some_list:
        tokenized_text = []
        tokenized_text.append(word_tokenize(string))
    tokenized_text = flatten_list(tokenized_text)
    clean_text = []
    filter = stoplist2 + more_stops
    filter.extend (list(more_punctuation))
    tokenized_text = [token.lower() for token in tokenized_text] 
    #counter = [n for n in num_most_freq]
    for i, token in enumerate(tokenized_text):
        if token not in filter:
            lemmatized_token = parser.parse(token)[0].normal_form
            clean_text.append(lemmatized_token)
            words_and_frequencies = Counter (clean_text).most_common(num_most_freq)
            freq_dict = {}
            i=0
            for pair in words_and_frequencies:
                i+=1
                freq_dict[i] = pair[0]
    return freq_dict

In [112]:
period1_records_keywords = keywords_most_frequent_lemma(period1_records_flat, 100)

In [115]:
period1_records_keywords

{1: 'письмо',
 2: 'что-то',
 3: 'думать',
 4: 'ответить',
 5: 'полный',
 6: 'вчера',
 7: 'проходить',
 8: 'подождать',
 9: '1',
 10: 'ч.',
 11: 'ночь',
 12: 'положительно',
 13: 'недоумевать',
 14: 'промолчать',
 15: 'такой',
 16: 'мой',
 17: '14740382394160',
 18: 'невозможно',
 19: 'особо',
 20: 'уважительный',
 21: 'причина',
 22: 'достаточно',
 23: 'фраза',
 24: 'пусть',
 25: 'несчастный',
 26: 'бесчестный',
 27: 'александр',
 28: 'михаилович',
 29: 'а.',
 30: 'м.',
 31: 'тотчас',
 32: 'основание',
 33: 'предполагать',
 34: 'сбыться',
 35: 'выпить',
 36: 'водка',
 37: 'сильно',
 38: 'подействовать',
 39: 'лечь',
 40: 'спать',
 41: 'полубесчувствие',
 42: 'больной',
 43: 'земляческий',
 44: 'собрание',
 45: 'ресторан',
 46: 'пить',
 47: 'толковать',
 48: 'оживление',
 49: 'роль',
 50: 'интеллигенция',
 51: 'отношение',
 52: 'рабочий',
 53: 'вопрос',
 54: 'подходить',
 55: 'дом',
 56: 'уверенность',
 57: 'ждать',
 58: 'сердце',
 59: 'замирать',
 60: 'готовиться',
 61: 'дурной',
 62: 

In [114]:
shared_items = {k: period1_records_keywords[k] for k in period1_records_keywords if k in larskaya_records_keywords and period1_records_keywords[k] == larskaya_records_keywords[k]}
print(shared_items)

{42: 'больной'}


In [108]:
larskaya_records_keywords = keywords_most_frequent_lemma(larskaya_records_flat, 100)

In [109]:
larskaya_records_keywords

{1: 'помочь',
 2: 'n.',
 3: 'филиппович',
 4: 'сон',
 5: 'любить',
 6: 'евгений',
 7: 'стих',
 8: 'свекровь',
 9: 'стать',
 10: 'рука',
 11: 'жизнь',
 12: 'кричать',
 13: 'пол',
 14: 'помыть',
 15: 'рассказать',
 16: 'умерший',
 17: 'депрессия',
 18: 'окно',
 19: 'шкаф',
 20: 'занавеска',
 21: 'быстро',
 22: 'бок',
 23: 'болеть',
 24: 'человек',
 25: 'помогать',
 26: 'писать',
 27: 'друг',
 28: 'познакомиться',
 29: 'и.',
 30: 'женщина',
 31: 'услышать',
 32: 'цветаев',
 33: 'сорок',
 34: 'пастернак',
 35: 'е.ф',
 36: 'любимый',
 37: 'умереть',
 38: 'долг',
 39: 'семья',
 40: 'затмение',
 41: 'зуб',
 42: 'больной',
 43: 'сбыться',
 44: 'сниться',
 45: 'какой-то',
 46: 'вещь',
 47: 'уронить',
 48: 'должный',
 49: 'постирать',
 50: 'германия',
 51: 'позвонить',
 52: 'хороший',
 53: 'приятельница',
 54: 'сложный',
 55: 'отношение',
 56: 'помолиться',
 57: 'упокоение',
 58: 'душа',
 59: 'простить',
 60: 'нелюбовь',
 61: 'связать',
 62: 'грязный',
 63: 'вата',
 64: 'мыть',
 65: 'легко',
 66

In [118]:
6 / 1301
#3 / 2207
#3 / 936

0.004611837048424289

In [179]:
import math
math.sqrt(0.6618994474)

0.8135720296322877

In [139]:
#пробуем искать паттерны с помощью регулярок
dictoinary = []
for item in female3_dict.items():
    #if re.search(r"настоящ.{2,9}\sлюб.{2,5}", str(item[1])):
    #if re.search(r"(разо(йтись|шлись|шлась|шелся|йдемся)|расста(ться|немся|нусь|лись|ешься)|расставани.{1,2})", str(item[1])):
    #if re.search(r"(не\sмогу|не\sспособен|не\sспособен)\s[а-я]+?любить", str(item[1])):
    #if re.search(r"(в?люб(ить|лю|ил|ила|иться)(сь)?\sпо-настоящему)|(по-настоящему\sв?люб(ить|лю|ил|ила|иться)(сь)?)", str(item[1])):
    #if re.search(r"трепет(но|ный|ного|ать|ал)?.{1,2}", str(item)):
    #if re.search(r"(за)?(при)?ревн(овать|овал|ревную|овала|ревновав)", str(item)):
    #if re.search(r"(лунн.{1,4}(свет|ноч).{0,4}|(свет|ноч).{0,5}лунн.{1,4})", str(item)):
    if re.search(r"(сердц.{1,10}б(ит|ил|ьё?е?).{0,5}|би(ит|ил|ьё?е?).{1,10}сердц.{0,5})", str(item)):
    #if re.search(r"дребедень", str(item)):
        #dictoinary.append(item)
#len(dictoinary)
        print(item)

(386871, ['Дорогой мой, если бы ты знал, какие дни я переживаю, что в душе моей и сердце, ты полюбил бы меня еще сильнее. Владик, как я люблю тебя, ты самый хороший, умный, бесценный. Одно мое желание — скорее встретиться с тобой, мы были бы теперь навсегда счастливы, но Боже сохрани, если разлука, тогда смерть мне. Проходят дни, годы, забываешь горе и снова будешь радоваться жизни; я не хочу этого, мое счастье только ты. Пусть смерть возьмет меня раньше, чем время залечит все душевные раны и возвратит мне покой.    \nЯ плачу, пусть с этими слезами исчезнет горе, ты вернешься, у тебя будет сын, Владя. Сколько радости нас ждет, вернись, мой желанный, вернись, мой родной!    \nПисем от Влади нет, что мне делать? Уже месяц — ни слова. За что мне столько мучений, столько страданий? Я всегда была добра к людям, за что меня наказывает Творец. в чем я грешна?'])
(213316, ['Суббота. После обеда, когда я уже поднималась по лестнице в цехе, меня окликнул Вовка с усиками и улыбаясь подал письмо. 

In [153]:
#period5_dict